In [3]:
import pandas as pd

dat10x = pd.read_csv("~/workspace/tcr-converter/example_10x.csv")
dat10x

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,AAACCTGAGACCACGA-1,True,AAACCTGAGACCACGA-1_contig_1,True,521,TRA,TRAV1-2,None,TRAJ12,TRAC,True,True,CAVMDSSYKLIF,TGTGCTGTGATGGATAGCAGCTATAAATTGATCTTC,1569,2,clonotype16,clonotype16_consensus_1
1,AAACCTGAGACCACGA-1,True,AAACCTGAGACCACGA-1_contig_3,True,584,TRB,TRBV6-1,TRBD2,TRBJ2-1,TRBC2,True,True,CASSGLAGGYNEQFF,TGTGCCAGCAGTGGACTAGCGGGGGGCTACAATGAGCAGTTCTTC,5238,7,clonotype16,clonotype16_consensus_2
2,AAACCTGAGGCTCTTA-1,True,AAACCTGAGGCTCTTA-1_contig_1,True,551,TRB,TRBV6-4,TRBD2,TRBJ2-3,TRBC2,True,True,CASSGVAGGTDTQYF,TGTGCCAGCAGTGGGGTAGCGGGAGGCACAGATACGCAGTATTTT,3846,4,clonotype26,clonotype26_consensus_2
3,AAACCTGAGGCTCTTA-1,True,AAACCTGAGGCTCTTA-1_contig_2,True,518,TRA,TRAV1-2,None,TRAJ33,TRAC,True,True,CAVKDSNYQLIW,TGTGCTGTGAAGGATAGCAACTATCAGTTAATCTGG,2019,2,clonotype26,clonotype26_consensus_1
4,AAACCTGAGTGAACGC-1,True,AAACCTGAGTGAACGC-1_contig_1,True,674,TRB,TRBV2,TRBD1,TRBJ1-2,TRBC1,True,True,CASNQGLNYGYTF,TGTGCCAGCAATCAGGGCCTTAACTATGGCTACACCTTC,3002,6,clonotype81,clonotype81_consensus_2


In [4]:
my_df = dat10x[['barcode', 'v_gene', 'j_gene', 'cdr3']]
my_df

,barcode,v_gene,j_gene,cdr3
0,AAACCTGAGACCACGA-1,TRAV1-2,TRAJ12,CAVMDSSYKLIF
1,AAACCTGAGACCACGA-1,TRBV6-1,TRBJ2-1,CASSGLAGGYNEQFF
2,AAACCTGAGGCTCTTA-1,TRBV6-4,TRBJ2-3,CASSGVAGGTDTQYF
3,AAACCTGAGGCTCTTA-1,TRAV1-2,TRAJ33,CAVKDSNYQLIW
4,AAACCTGAGTGAACGC-1,TRBV2,TRBJ1-2,CASNQGLNYGYTF


In [5]:
col_ref = {'tenx': ('v_gene', 'j_gene', 'cdr3'),
            'imgt': ('v_gene', 'j_gene', 'cdr3'),
            'adaptive': ('v_resolved', 'j_resolved', 'cdr3_amino_acid'),
            'adaptive_v2': ('vMaxResolved', 'jvMaxResolved', 'aminoAcid')}


lookup_dict = {
    'tenx': ['TRAV1-2', 'TRBV6-1', 'TRBV6-4', 'TRBV2', 'TRBV9', 'TRAJ12', 
                'TRBJ2-1', 'TRBJ2-3', 'TRAJ33', 'TRBJ1-2'],
    'imgt': ['TRAV1-2*01', 'TRBV6-1*01', 'TRBV6-4*01', 'TRBV2*01', 'TRBV9*01', 'TRAJ12*01', 
                'TRBJ2-1*01', 'TRBJ2-3*01', 'TRAJ33*01', 'TRBJ1-2*01'],
    'adaptive': ['TCRAV01-02*01', 'TCRBV06-01*01', 'TCRBV06-04*01', 'TCRBV02*01',
                    'TCRBV09*01', 'TCRAJ12*01', 'TCRBJ02-01*01', 'TCRBJ02-03*01',
                    'TCRAJ33*01', 'TCRBJ01-02*01'],
    'adaptive_v2': ['TCRAV01-02*01', 'TCRBV06-01*01', 'TCRBV06-04*01', 'TCRBV02*01',
                    'TCRBV09*01', 'TCRAJ12*01', 'TCRBJ02-01*01', 'TCRBJ02-03*01',
                    'TCRAJ33*01', 'TCRBJ01-02*01']
}

lookup = pd.DataFrame(lookup_dict)

lookup

,tenx,imgt,adaptive,adaptive_v2
0,TRAV1-2,TRAV1-2*01,TCRAV01-02*01,TCRAV01-02*01
1,TRBV6-1,TRBV6-1*01,TCRBV06-01*01,TCRBV06-01*01
2,TRBV6-4,TRBV6-4*01,TCRBV06-04*01,TCRBV06-04*01
3,TRBV2,TRBV2*01,TCRBV02*01,TCRBV02*01
4,TRBV9,TRBV9*01,TCRBV09*01,TCRBV09*01
5,TRAJ12,TRAJ12*01,TCRAJ12*01,TCRAJ12*01
6,TRBJ2-1,TRBJ2-1*01,TCRBJ02-01*01,TCRBJ02-01*01
7,TRBJ2-3,TRBJ2-3*01,TCRBJ02-03*01,TCRBJ02-03*01
8,TRAJ33,TRAJ33*01,TCRAJ33*01,TCRAJ33*01
9,TRBJ1-2,TRBJ1-2*01,TCRBJ01-02*01,TCRBJ01-02*01


In [6]:
def convert_tcr(df, 
                fmt_from=['tenx', 'adaptive', 'adaptive_v2', 'imgt'],
                fmt_to=['tenx', 'adaptive', 'adaptive_v2', 'imgt'],
                cols_use=None, extract_tcr=False, change_colname=False):
    # cols_use: Tuple of v, j, and cdr3 columns names to pull from
    # extract_tcr: Extract just tcr columns (v, j, cdr3) from the dataframe
    # change_colname: Use column names that match the output format

    # Figure out columns to use
    if cols_use:
        cols_from = cols_use
    elif fmt_from == 'adaptive' and 'vMaxResolved' in df.columns:
        cols_from = 'adaptive_v2'
    elif fmt_from == 'imgt' and not cols_use:
        print('No column names provided for IMGT data, will use 10X column names:\n' + str(col_ref['tenx']))
        cols_from = 'tenx'
    else:
        cols_from = fmt_from

    # Column names to use for input and output
    v_from = col_ref[cols_from][0]
    j_from = col_ref[cols_from][1]
    cdr3_from = col_ref[cols_from][2]

    if change_colname:
        v_to = col_ref[fmt_to][0]
        j_to = col_ref[fmt_to][1]
        cdr3_to = col_ref[fmt_to][2]
    else:
        v_to = v_from
        j_to = j_from
        cdr3_to = cdr3_from

    # Convert
    out = pd.DataFrame()
    out[v_to] = df[v_from].map(lookup.set_index(fmt_from)[fmt_to])
    out[j_to] = df[j_from].map(lookup.set_index(fmt_from)[fmt_to])
    out[cdr3_to] = df[cdr3_from]

    if extract_tcr:
        return out
    else:
        out_df = df.copy()
        out_df[v_from] = out[v_to].values
        out_df[j_from] = out[j_to].values
        out_df[cdr3_from] = out[cdr3_to].values
        out_df.rename(columns={v_from: v_to, 
                               j_from: j_to,
                               cdr3_from: cdr3_to}, inplace = True)
        return out_df


In [7]:
my_df

,barcode,v_gene,j_gene,cdr3
0,AAACCTGAGACCACGA-1,TRAV1-2,TRAJ12,CAVMDSSYKLIF
1,AAACCTGAGACCACGA-1,TRBV6-1,TRBJ2-1,CASSGLAGGYNEQFF
2,AAACCTGAGGCTCTTA-1,TRBV6-4,TRBJ2-3,CASSGVAGGTDTQYF
3,AAACCTGAGGCTCTTA-1,TRAV1-2,TRAJ33,CAVKDSNYQLIW
4,AAACCTGAGTGAACGC-1,TRBV2,TRBJ1-2,CASNQGLNYGYTF


In [8]:
new_df = convert_tcr(my_df, fmt_from='tenx', fmt_to='adaptive', extract_tcr=False, change_colname=True)
new_df

,barcode,v_resolved,j_resolved,cdr3_amino_acid
0,AAACCTGAGACCACGA-1,TCRAV01-02*01,TCRAJ12*01,CAVMDSSYKLIF
1,AAACCTGAGACCACGA-1,TCRBV06-01*01,TCRBJ02-01*01,CASSGLAGGYNEQFF
2,AAACCTGAGGCTCTTA-1,TCRBV06-04*01,TCRBJ02-03*01,CASSGVAGGTDTQYF
3,AAACCTGAGGCTCTTA-1,TCRAV01-02*01,TCRAJ33*01,CAVKDSNYQLIW
4,AAACCTGAGTGAACGC-1,TCRBV02*01,TCRBJ01-02*01,CASNQGLNYGYTF
